<a href="https://colab.research.google.com/github/k-js/analise-de-ETFs-/blob/main/an%C3%A1lise_de_ETFs_(Exchange_Traded_Funds)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Python para Finanças </h1>

<h2> Lista de Exercícios 3 </h2>


**Os exercícios devem ser feitos em python, podendo ser feito em Excel apenas para conferência dos resultados obtidos.**

**A lista deve ser entregue até às 23:59 do dia 21/05/2025, horário de Brasília.**

Para entrega, o aluno deve entregar o arquivo jupyter (.ipynb) fornecido pelo professor devidamente preenchido com as respostas das questões. A entrega deve ser feita para o e-mail do professor (jorge.antunes@pep.ufrj.br) e do coordenador do curso (rafael.cardoso@chcfinance.com.br).

Cada questão tem sua pontuação no enunciado e a lista soma 10 pontos no total. Se atente que algumas questões são encadeadas, o que significa que o resultado de uma questão afeta a seguinte, dessa forma, procure ter certeza de que o resultado está realmente correto para sua nota não ser prejudicada por um erro no começo.

Boa sorte!



<h4>Preparação dos pacotes e diretórios</h4>

In [ ]:
## Importar as bibliotecas necessárias

In [ ]:
from google.colab import files
import pandas as pd
import yfinance as yf
import numpy as np
from sklearn.linear_model import LinearRegression

<h3>Questão 1</h3>
<h4>Pontuação: 2 pts</h4>

Você foi alocado na equipe de análise de ETFs (Exchange Traded Funds) do banco onde trabalha. Atualmente, os processos são realizados de forma manual em Excel, exigindo o esforço repetitivo de colaboradores para coletar informações diretamente dos sites de mercado.

Para melhorar a eficiência, desenvolva um programa em Python com as seguintes funcionalidades:
1) Leia uma lista de ETFs atualizada da B3 (busque no site da B3 este arquivo e faça seu download, o código não precisa baixar o arquivo, apenas ler ele).
2) Utilize uma API adequada para baixar os preços de fechamento desses ETFs de forma automática.
3) Utilize uma API adequada para baixar os preços do índice Bovespa (Benchmark)
4) Utilize uma API adequada para baixar os preços do CDI (Risk Free).

Considere como data inicial 2024-01-01 e como data final 2025-04-30.

Os dados dos ETFs devem ser dispostos como DataFrame e devem conter apenas ETFs com dados completos neste período.

Deixe todas as datas no formato YYYY-MM-DD.

In [ ]:
# Caminho para o arquivo CSV da B3 com ETFs (você deve colocar o caminho correto)
file_path = "/content/fundosListados.csv"
# Carregando o arquivo CSV
df_etfs = pd.read_csv(file_path, sep = ";", encoding = "latin1", header = 0, index_col = False)
print(df_etfs.head())

# Gerando os tickers
tickers = df_etfs["Código"].dropna().astype(str).str.strip().str.upper().add("11.SA").tolist()

print(f"\n✅ Total de Tickers encontrados: {len(tickers)}")

# Criando um novo DataFrame com os tickers
df_tickers = pd.DataFrame({"Tickers": tickers})

# Exibindo o novo DataFrame
print(df_tickers.head())

                                        Razão Social         Fundo Código
0                BB ETF IAGRO-FFS B3 FUNDO DE ÍNDICE  BB ETF IAGRO   AGRI
1                    BB ETF IBOVESPA FUNDO DE INDICE   BB ETF IBOV   BBOV
2  BB ETF ÍNDICE BOVESPA B3 BR+ FUNDO DE ÍNDICE R...   BB ETF BRAZ   BRAZ
3  BB ETF ÍNDICE DIVERSIDADE B3 INVESTIMENTO SUST...   BB ETF DVER   DVER
4  BB ETF ÍNDICE FUTURO DE BOI GORDO B3 FUNDO DE ...  BB ETF BOI G   BBOI

✅ Total de Tickers encontrados: 86
     Tickers
0  AGRI11.SA
1  BBOV11.SA
2  BRAZ11.SA
3  DVER11.SA
4  BBOI11.SA


In [ ]:
tickers_validos = []
tickers_sem_preco = []
tickers_invalidos = []

for ticker in tickers:
    try:
        data = yf.Ticker(ticker)
        # Verifica se há preço de mercado regular disponível
        if data.info.get('regularMarketPrice') is not None:
            #print(f"{ticker} é válido.")
            tickers_validos.append(ticker)
        else:
            #print(f"{ticker} não possui preço de mercado.")
            tickers_sem_preco.append(ticker)
    except Exception as e:
        print(f"Erro ao buscar {ticker}: {e}")
        tickers_invalidos.append(ticker)

print("\n=== RESUMO ===")
print(f"Total de tickers testados: {len(tickers)}")
print(f"Tickers válidos: {len(tickers_validos)}")
print(f"Tickers sem preço de mercado: {len(tickers_sem_preco)}")
print(f"Tickers inválidos: {len(tickers_invalidos)}")

# salvar as listas em arquivo para análise futura:
with open("tickers_validos.txt", "w") as f_val:
    for t in tickers_validos:
        f_val.write(t + "\n")

with open("tickers_invalidos.txt", "w") as f_inv:
    for t in tickers_invalidos:
        f_inv.write(t + "\n")

# Intervalo de datas
data_inicial = "2024-01-01"
data_final = "2025-04-30"

# Lista para armazenar os DataFrames de cada ETF
dfs = []

# Loop para buscar os dados de cada ETF
for ticker in tickers_validos:
    try:
        # Criando o objeto Ticker
        data = yf.Ticker(ticker)

        # Obtendo os dados históricos no intervalo de datas
        # Allow for potential incomplete data ranges
        historico = data.history(start=data_inicial, end=data_final)

        # Verificando se há dados disponíveis
        if len(historico) > 0: # Check if there is any data
            # Adicionando o ticker como uma coluna no DataFrame
            historico['Ticker'] = ticker

            # Adicionando ao conjunto de DataFrames
            dfs.append(historico[['Ticker', 'Close']])
        else:
            # Opcional: Imprimir uma mensagem se não houver dados para o ticker no intervalo
            print(f"Nenhum dado histórico encontrado para o ticker {ticker} no intervalo especificado.")

    except Exception as e:
        print(f"Erro ao buscar dados para o ticker {ticker}: {e}")


if len(dfs) > 0:
    # Concatenando todos os DataFrames em um único DataFrame
    df_final = pd.concat(dfs)

    # Formatando o índice de datas para o formato YYYY-MM-DD
    df_final.index = df_final.index.strftime('%Y-%m-%d')

    # Resetando o índice para exibir as datas como uma coluna
    df_final.reset_index(inplace=True)
    df_final.rename(columns={"index": "Date"}, inplace=True)

    # Exibindo o DataFrame final
    print(df_final)
else:
    print("Nenhum DataFrame foi coletado. df_final está vazio.")


Erro ao buscar CASA11.SA: HTTP Error 404: 
Erro ao buscar GPUS11.SA: HTTP Error 404: 


ERROR:yfinance:$BRAZ11.SA: possibly delisted; no price data found  (1d 2024-01-01 -> 2025-04-30)



=== RESUMO ===
Total de tickers testados: 86
Tickers válidos: 82
Tickers sem preço de mercado: 2
Tickers inválidos: 2
Nenhum dado histórico encontrado para o ticker BRAZ11.SA no intervalo especificado.


ERROR:yfinance:$DOLA11.SA: possibly delisted; no price data found  (1d 2024-01-01 -> 2025-04-30) (Yahoo error = "Data doesn't exist for startDate = 1704078000, endDate = 1745982000")
ERROR:yfinance:$TECX11.SA: possibly delisted; no price data found  (1d 2024-01-01 -> 2025-04-30)
ERROR:yfinance:$PKIN11.SA: possibly delisted; no price data found  (1d 2024-01-01 -> 2025-04-30)


Nenhum dado histórico encontrado para o ticker DOLA11.SA no intervalo especificado.
Nenhum dado histórico encontrado para o ticker TECX11.SA no intervalo especificado.
Nenhum dado histórico encontrado para o ticker PKIN11.SA no intervalo especificado.


...

Nenhum dado histórico encontrado para o ticker SPXR11.SA no intervalo especificado.


ERROR:yfinance:$HIGH11.SA: possibly delisted; no price data found  (1d 2024-01-01 -> 2025-04-30)
ERROR:yfinance:$LVOL11.SA: possibly delisted; no price data found  (1d 2024-01-01 -> 2025-04-30)
ERROR:yfinance:$SPUB11.SA: possibly delisted; no price data found  (1d 2024-01-01 -> 2025-04-30)


Nenhum dado histórico encontrado para o ticker HIGH11.SA no intervalo especificado.
Nenhum dado histórico encontrado para o ticker LVOL11.SA no intervalo especificado.
Nenhum dado histórico encontrado para o ticker SPUB11.SA no intervalo especificado.


ERROR:yfinance:$SPVT11.SA: possibly delisted; no price data found  (1d 2024-01-01 -> 2025-04-30)


Nenhum dado histórico encontrado para o ticker SPVT11.SA no intervalo especificado.
             Date     Ticker      Close
0      2024-01-02  AGRI11.SA  50.700001
1      2024-01-03  AGRI11.SA  50.660000
2      2024-01-04  AGRI11.SA  49.889999
3      2024-01-05  AGRI11.SA  50.340000
4      2024-01-08  AGRI11.SA  50.980000
...           ...        ...        ...
20586  2025-04-23  NASD11.SA  14.890000
20587  2025-04-24  NASD11.SA  15.230000
20588  2025-04-25  NASD11.SA  15.390000
20589  2025-04-28  NASD11.SA  15.270000
20590  2025-04-29  NASD11.SA  15.350000

[20591 rows x 3 columns]


In [ ]:
## Leitura do benchmark

# Define o benchmark (índice Bovespa)
benchmark = "^BVSP"

# Cria a lista total de tickers incluindo o benchmark
todos_tickers = tickers + [benchmark]

# Baixar os dados de fechamento
dados = yf.download(todos_tickers, start=data_inicial, end=data_final)["Close"]

# Separar benchmark
dados_benchmark = dados[benchmark]

# Remover benchmark dos ETFs
dados_etfs = dados.drop(columns=[benchmark])

print(f"Benchmark (Bovespa):\n{dados_benchmark.head()}")

[*********************100%***********************]  87 of 87 completed
ERROR:yfinance:
16 Failed downloads:
ERROR:yfinance:['SPUB11.SA', 'SPVT11.SA', 'HIGH11.SA', 'BRAZ11.SA', 'PKIN11.SA', 'TECX11.SA', 'B3BR11.SA', 'SPXR11.SA', 'LVOL11.SA']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-01-01 -> 2025-04-30)')
ERROR:yfinance:['BIZD11.SA', 'DOLA11.SA', 'NUCL11.SA']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-01-01 -> 2025-04-30) (Yahoo error = "Data doesn\'t exist for startDate = 1704078000, endDate = 1745982000")')
ERROR:yfinance:['GPUS11.SA', 'CASA11.SA', 'ESGU11.SA', 'URET11.SA']: YFTzMissingError('possibly delisted; no timezone found')


Benchmark (Bovespa):
Date
2024-01-02    132697.0
2024-01-03    132834.0
2024-01-04    131226.0
2024-01-05    132023.0
2024-01-08    132427.0
Name: ^BVSP, dtype: float64


In [ ]:
## Leitura do CDI

# Parâmetros
ini, fim = "01/01/2024", "30/04/2025"
formato = "json"
serie = 12  # CDI

url = (f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{serie}/dados?"
       f"formato={formato}&dataInicial={ini}&dataFinal={fim}")

cdi = (
    pd.read_json(url)
    .rename(columns = {"data": "Date","valor": "CDI"})
    .assign(Date = lambda x: pd.to_datetime(x["Date"], format = "%d/%m/%Y"))
    .set_index("Date")
    .reindex(dados.index)  # Changed from data_tickers.index to dados.index
)

# Import numpy for cumprod
import numpy as np
cdi["CDI"] = np.cumprod(1 + cdi["CDI"] / 100)
print(f"DataFrame CDI:\n{cdi.head(10)}\n")

# Limpeza: remover ETFs (colunas) com qualquer valor ausente
dados_completos = dados.dropna(axis=1, how="any")

# Atualiza a versão de ETFs apenas com os que passaram na limpeza
dados_etfs = dados_completos.drop(columns=[benchmark])

# Verificação final antes de salvar
if not dados_completos.empty and dados_etfs.shape[1] > 0:
    output_file = "dados_etfs_bvsp_limpo.xlsx"
    dados_completos.to_excel(output_file)
    print(f"✅ {dados_etfs.shape[1]} ETFs com dados completos salvos.")
    print(f"📊 Total de colunas no Excel (inclui benchmark): {dados_completos.shape[1]}")
    # Import and use files.download
    try:
        from google.colab import files
        files.download(output_file)
    except ImportError:
        print("Rodando fora do Google Colab, files.download não disponível.")
    print(f"Arquivo salvo com apenas ETFs completos: {output_file}", "sucesso")
else:
    print("❌ Nenhum ETF com dados completos disponível. Nada foi salvo.", "erro")

DataFrame CDI:
                 CDI
Date                
2024-01-02  1.000437
2024-01-03  1.000875
2024-01-04  1.001313
2024-01-05  1.001751
2024-01-08  1.002189
2024-01-09  1.002627
2024-01-10  1.003066
2024-01-11  1.003504
2024-01-12  1.003943
2024-01-15  1.004383

✅ 34 ETFs com dados completos salvos.
📊 Total de colunas no Excel (inclui benchmark): 35


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Arquivo salvo com apenas ETFs completos: dados_etfs_bvsp_limpo.xlsx sucesso


<h3>Questão 2</h3>
<h4>Pontuação: 3 pts</h4>

Após o download, utilize os dados do período entre 01/01/2024 e 30/04/2025 para calcular o retorno simples de cada ETF, do índice Bovespa e do CDI.

Antes de realizar o cálculo:

1) Certifique-se de que os nomes dos índices (index) e das colunas (columns) estejam corretos e façam sentido.
2) Verifique se as datas são coincidentes entre os três DataFrames. Caso contrário, realize os ajustes necessários.
3) Ordene os dados por data.
4) Verifique se os valores dos dados necessitam de transformação


In [ ]:
# Calcular o retorno simples total para cada ativo/índice (preço final / preço inicial - 1)
retorno_simples_total = (dados.iloc[-1] / dados.iloc[0]) - 1

# Calcular o retorno simples do CDI (mesma lógica)
retorno_cdi = (cdi["CDI"].iloc[-1] / cdi["CDI"].iloc[0]) - 1

# Adicionar o CDI ao DataFrame de retornos
retorno_simples_total["CDI"] = retorno_cdi

# Remover NaNs (caso existam)
retorno_simples_total = retorno_simples_total.dropna()

# Converter para porcentagem e formatar com 2 casas decimais
retorno_simples_pct = (retorno_simples_total * 100).round(2)

# Extrair só o prefixo do ticker
def extrair_prefixo(ticker):
    if "11.SA" in ticker:
        return ticker.split("11.SA")[0]
    elif ".SA" in ticker:
        return ticker.split(".SA")[0]
    else:
        return ticker

retorno_simples_pct.index = retorno_simples_pct.index.map(extrair_prefixo)
retorno_simples_pct.rename(index={"^BVSP": "IBOV"}, inplace=True)

# Mostrar o resultado
print(f"Retorno simples de cada ativo e índice ({data_inicial} a {data_final}):")
for ativo, retorno in retorno_simples_pct.items():
    print(f"{ativo}: {retorno:.2f}%")


Retorno simples de cada ativo e índice (2024-01-01 a 2025-04-30):
ACWI: 33.89%
AGRI: -4.71%
ALUG: 16.42%
BBOV: 2.72%
BBSD: 0.02%
BOVA: 2.41%
BOVB: 2.43%
BOVS: 1.91%
BOVV: 2.36%
BOVX: 2.46%
BRAX: 2.43%
CHIP: 8.77%
CMDB: -2.83%
DIVO: 7.70%
ECOO: 11.58%
ELAS: 1.39%
ESGB: -3.11%
FIND: 9.37%
GENB: 77.79%
GLDX: 11.59%
GOLD: 83.51%
GOVE: 2.07%
HTEK: -2.60%
IBOB: 2.70%
ISUS: -0.92%
IVVB: 35.37%
JOGO: 91.84%
MATB: -16.67%
MILL: 36.45%
NASD: 35.48%
PIBB: 3.43%
REVE: 16.90%
SCVB: -7.41%
SMAB: -9.08%
SMAC: -8.09%
SMAL: -8.00%
SPXB: 34.22%
SPXI: 35.24%
SVAL: 3.88%
TECK: 66.00%
TRIG: -1.27%
USAL: 35.94%
USTK: 32.63%
UTEC: 33.43%
WRLD: 31.86%
XBOV: 1.79%
XINA: 51.12%
YDRO: -16.38%
IBOV: 1.81%
CDI: 15.18%


<h3>Questão 3</h3>
<h4>Pontuação: 5 pts</h4>

Agora que os dados de retornos dos ETFs, do CDI e do índice Bovespa (benchmark) foram baixados e devidamente tratados, desenvolva um código que compute as seguintes métricas de desempenho para cada ETF:

1) Volatilidade
2) Índice de Sharpe
3) Indice de Sortino
4) Value at Risk (VaR) histórico com 95% de confiança
5) Beta
6) Alfa

O resultado deves ser apresentado em um data frame com as seguintes colunas:
1) Nome do ativo
2) Retorno médio do ativo
3) Volatilidade
4) Índice de Sharpe
5) Indice de Sortino
6) Value at Risk (VaR) histórico
7) Beta
8) Alfa

<span style="color:red">Imprima o resultado do DataFrame no console do notebook mostrando todas as linhas e colunas.</span> Depois, salve o DataFrame em um arquivo Excel.


O Índice de Sortino é calculado pela fórmula:

$$
Sortino = \frac{\overline{r_{i} - r_{rf}}}{DR}
$$

Onde:

${r_i}$ é o retorno do ativo,

$r_{rf}$ é o retorno do ativo livre de risco (CDI),

$DR$ é o downside deviation, dado por:

$$DR = \sqrt{\frac{\sum_{i=1}^{n}{\min(r_i - r_{rf}, 0)^2}}{n-1}}$$



In [ ]:
# === 1. Cálculo dos retornos diários simples ===
retornos_etfs = dados_etfs.pct_change().dropna()
retorno_benchmark = dados_benchmark.pct_change().dropna()
retorno_cdi = cdi["CDI"].pct_change().dropna()

# === 2. Unificação e alinhamento dos dados ===
df_completo = retornos_etfs.copy()
df_completo["Benchmark"] = retorno_benchmark
df_completo["CDI"] = retorno_cdi

df_completo = df_completo.dropna()  # garante alinhamento temporal

# === 3. Separar colunas de interesse ===
col_etfs = retornos_etfs.columns.tolist()
ret_bench = df_completo["Benchmark"]
ret_rf = df_completo["CDI"]
dados_etfs_alinhados = df_completo[col_etfs]

# === 4. Funções estatísticas ===
def sharpe(x, rf):
    excesso = x - rf
    desv = np.std(excesso, ddof=1)
    return np.mean(excesso) / desv if desv != 0 else np.nan

def sortino(x, rf):
    excesso = x - rf
    downside = np.minimum(excesso, 0)
    dr = np.sqrt(np.sum(downside ** 2) / (len(x) - 1)) if len(x) > 1 else np.nan
    return np.mean(excesso) / dr if dr != 0 else np.nan

def VaR_historico(x, ic=0.95):
    return np.quantile(x.dropna(), 1 - ic) if len(x.dropna()) > 0 else np.nan

def beta(x, b, rf):
    excesso_x = x - rf
    excesso_b = b - rf
    cov = np.cov(excesso_x, excesso_b, ddof=1)[0, 1]
    var = np.var(excesso_b, ddof=1)
    return cov / var if var != 0 else np.nan

def alfa(x, b, rf):
    excesso_x = np.mean(x - rf)
    excesso_b = np.mean(b - rf)
    beta_val = beta(x, b, rf)
    return excesso_x - beta_val * excesso_b if not np.isnan(beta_val) else np.nan

# === 5. Cálculo das métricas ===
resultados = []

for ativo in dados_etfs_alinhados.columns:
    x = dados_etfs_alinhados[ativo]

    if x.dropna().shape[0] < 30:
        continue  # pula ETF com poucos dados

    resultados.append({
        "Ativo": ativo,
        "Retorno médio": np.mean(x),
        "Volatilidade": np.std(x, ddof=1),
        "Índice de Sharpe": sharpe(x, ret_rf),
        "Índice de Sortino": sortino(x, ret_rf),
        "VaR Histórico": VaR_historico(x),
        "Beta": beta(x, ret_bench, ret_rf),
        "Alfa": alfa(x, ret_bench, ret_rf)
    })

# === 6. Exportar e exibir ===
df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel("metricas_etfs.xlsx", index=False)
print(df_resultados)


        Ativo  Retorno médio  Volatilidade  Índice de Sharpe  \
0   ACWI11.SA       0.000933      0.009826          0.051300   
1   ALUG11.SA       0.000544      0.012922          0.008959   
2   BBOV11.SA       0.000123      0.009143         -0.033417   
3   BOVA11.SA       0.000115      0.009228         -0.034024   
4   BOVB11.SA       0.000114      0.009045         -0.034810   
5   BOVS11.SA       0.000098      0.009002         -0.036753   
6   BOVV11.SA       0.000112      0.009128         -0.034652   
7   BOVX11.SA       0.000117      0.009319         -0.033429   
8   BRAX11.SA       0.000111      0.008747         -0.036316   
9   CMDB11.SA      -0.000023      0.011348         -0.039752   
10  DIVO11.SA       0.000259      0.008235         -0.020627   
11  FIND11.SA       0.000332      0.011031         -0.008729   
12  GENB11.SA       0.001898      0.017535          0.083812   
13  GOLD11.SA       0.001899      0.010740          0.136910   
14  ISUS11.SA       0.000031      0.0108